In [65]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
import pymongo
import datetime
import matplotlib.pyplot as plt

In [9]:
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv')
data = pd.read_csv(url, sep=',', header=0, names=['y', 'date'])
data['date'] = pd.to_datetime(data['date'], format='%Y/%m/%d')
data = data.set_index('date')
data = data.asfreq('MS')
data

,y
date,
1991-07-01,0.429795
1991-08-01,0.400906
1991-09-01,0.432159
1991-10-01,0.492543
1991-11-01,0.502369
...,...
2008-02-01,0.761822
2008-03-01,0.649435
2008-04-01,0.827887


In [7]:
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags = 5
             )

forecaster.fit(y=data['y'])

In [8]:
forecaster.predict(steps=5)

2008-07-01    0.714526
2008-08-01    0.789144
2008-09-01    0.818433
2008-10-01    0.845027
2008-11-01    0.914621
Freq: MS, Name: pred, dtype: float64

In [25]:
def calcPercentage(ls):
    sum=0
    for j in ls:
        sum+=(j/len(ls))/5
    return sum

In [59]:
def makeProb(listNutricional,listAfectivo,listFuncional,id):
    date = datetime.datetime.now().strftime("%d-%m-%Y")
    valueArray=np.array([1,2,3,4,5])
    converterNutri= [valueArray[-i] for i in listNutricional]
    converterAfect= [valueArray[-i] for i in listAfectivo]
    converterFunc= [valueArray[-i] for i in listFuncional]
    return {"id":id,"Nutritional":[{"date":date,"value":calcPercentage(converterNutri)}],"Affect":[{"date":date,"value":calcPercentage(converterAfect)}],"Func":[{"date":date,"value":calcPercentage(converterFunc)}]}

print(makeProb([5,5,5,5,5],[2,3,2,4,5],[3,3,1,2,3],"111"))


{'id': '111', 'Nutritional': [{'date': '15-10-2022', 'value': 0.2}], 'Affect': [{'date': '15-10-2022', 'value': 0.56}], 'Func': [{'date': '15-10-2022', 'value': 0.72}]}


In [64]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["local"]
collection = mydb["hosp"]
def addOne(req):
    old=collection.find_one({"id":req["id"]})
    if(old is None):
        collection.insert_one(req)
    else:
        old["Nutritional"].append(req["Nutritional"][0])
        old["Affect"].append(req["Affect"][0])
        old["Func"].append(req["Func"][0])
        collection.update_one({"id":req["id"]},{"$set":{"Nutritional":old["Nutritional"],"Affect":old["Affect"],"Func":old["Func"]}})
addOne(makeProb([5,5,5,5,5],[2,3,2,4,5],[3,3,1,2,3],"1111"))


In [68]:
data=collection.find_one({"id":"1111"})
print(data)
date=[d["date"] for d in data["Nutritional"]]
plt.plot(data["Nutritional"]["date"],data["Nutritional"]["value"])


{'_id': ObjectId('634b879a86baf58615ffe1b3'), 'id': '1111', 'Nutritional': [{'date': '15-10-2022', 'value': 0.2}], 'Affect': [{'date': '15-10-2022', 'value': 0.56}], 'Func': [{'date': '15-10-2022', 'value': 0.72}]}


TypeError: list indices must be integers or slices, not str